In [57]:
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from random import gauss
from mpl_toolkits.mplot3d import Axes3D
from scipy import stats as stats
from statsmodels.formula.api import ols

%matplotlib inline

In [58]:
import pandas as pd
kc_house_data_df = pd.read_csv('data/kc_house_data.csv')
kk = kc_house_data_df.copy()

In [59]:
kk['grade_num'] = kk['grade'].str.split()
kk['grade_num'] = kk['grade_num'].str[0]
kk['grade_num'] = kk['grade_num'].astype(int)
kk['grade_num'].min()

3

In [62]:
y = kk['price']
X = kk.drop('price', axis =1)

In [64]:
bedrooms_train = kk[['bedrooms']]
ohe_bedrooms = OneHotEncoder(categories ='auto', sparse =False)

In [65]:
ohe_bedrooms.fit(bedrooms_train)

OneHotEncoder(sparse=False)

In [66]:
ohe_bedrooms.categories_

[array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 33], dtype=int64)]

In [67]:
ohe_bedrooms_encoded = ohe_bedrooms.transform(bedrooms_train)
ohe_bedrooms_encoded

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [68]:
ohe_bedrooms_encoded = pd.DataFrame(ohe_bedrooms_encoded, columns = ohe_bedrooms.categories_[0], index = kk.index )
ohe_bedrooms_encoded

,1,2,3,4,5,6,7,8,9,10,11,33
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
21592,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21593,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21594,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21595,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
# kk.drop('bedrooms', axis= 1, inplace=True)
kk = pd.concat([kk, ohe_bedrooms_encoded ], axis =1)
# kk.drop('11', axis =1, inplace =True)

In [70]:
kk.drop(11, axis =1, inplace = True)
kk.drop(10, axis =1, inplace = True)
kk.drop(9, axis = 1, inplace =True)
kk.drop(33, axis = 1, inplace= True)

In [71]:
kk

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_lot15,grade_num,1,2,3,4,5,6,7,8
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,NONE,...,5650,7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,NO,NONE,...,7639,7,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,NO,NONE,...,8062,6,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,NO,NONE,...,5000,7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,NO,NONE,...,7503,8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,NO,NONE,...,1509,8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,NO,NONE,...,7200,8,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,NO,NONE,...,2007,7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,NONE,...,1287,8,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [72]:
grade_num_train = kk[['grade_num']]
ohe_grade_num = OneHotEncoder(categories ='auto', sparse =False)
ohe_grade_num.fit(grade_num_train)
ohe_grade_num.categories_

[array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])]

In [73]:
ohe_grade_num_encoded = ohe_grade_num.transform(grade_num_train)
ohe_grade_num_encoded = pd.DataFrame(ohe_grade_num_encoded, columns = ohe_grade_num.categories_[0], index = kk.index )

In [74]:
kk = pd.concat([kk, ohe_grade_num_encoded ], axis =1)

In [75]:
kk.drop('bedrooms', axis =1, inplace=True)
kk.drop('grade_num', axis =1, inplace =True)
kk.drop('grade', axis =1, inplace=True)
kk.head()

,id,date,price,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,4,5,6,7,8,9,10,11,12,13
0,7129300520,10/13/2014,221900.0,1.00,1180,5650,1.0,NaN,NONE,Average,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,6414100192,12/9/2014,538000.0,2.25,2570,7242,2.0,NO,NONE,Average,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5631500400,2/25/2015,180000.0,1.00,770,10000,1.0,NO,NONE,Average,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2487200875,12/9/2014,604000.0,3.00,1960,5000,1.0,NO,NONE,Very Good,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1954400510,2/18/2015,510000.0,2.00,1680,8080,1.0,NO,NONE,Average,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# df['year'] = df['ArrivalDate'].dt.year


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.shape[0], y_train.shape[0]

In [ ]:
kk['id'].value_counts()  #Seeing how many times individual houses were sold

In [ ]:
kk['condition'].value_counts()

In [ ]:
kk.loc[kk['id'] ==795000620, ['price'] ] #seeing price history for most common house id number

In [ ]:
kk['grade_num'] = kk['grade'].str.split()
kk['grade_num'] = kk['grade_num'].str[0]
kk['grade_num'] = kk['grade_num'].astype(int)
kk.head()
# Turning grade into just the int value

In [ ]:
kk['bathrooms'].value_counts()

In [ ]:
kk['ArrivalDate']=pd.to_datetime(kk['date'], format='%m/%d/%Y')
kk.head()

In [ ]:
kk['yr_renovated'].value_counts()

In [ ]:
kk['sqft_basement'].value_counts()

In [ ]:
kk.loc[kk['id'] ==795000620, ['grade#'] #seeing the grade for the most common listing

In [ ]:
kk['view'].value_counts()

In [ ]:
# all the pieces in one formula
f1 = 'price ~   C(bedrooms) + C(bathrooms) + sqft_living + sqft_lot+ grade_num + floors +C(waterfront) + C(view) + C(condition) + sqft_above + sqft_basement+ C(yr_built) +C(yr_renovated) + C(zipcode) + lat +long + sqft_living15 + sqft_lot15'
f1_model = ols(formula = f1, data =kk).fit()
f1_model_summary= f1_model.summary()
f1_model_summary

In [ ]:
#just doing grade_num to price
f2 = 'price~ grade_num'
f2_model = ols(formula =f2, data =kk).fit()
f2_model_summary = f2_model.summary()
f2_model_summary


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,3))

for xcol, ax in zip(['sqft_above', 'sqft_lot', 'sqft_living', 'sqft_living15'], axes):
    kk.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.4, color='b')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,3))

for xcol, ax in zip(['bedrooms', 'bathrooms', 'condition', 'grade'], axes):
    kk.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.4, color='b')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,3))

for xcol, ax in zip(['sqft_basement', 'yr_built', 'yr_renovated', 'zipcode'], axes):
    kk.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.4, color='b')

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,3))

for xcol, ax in zip(['sqft_lot15', 'floors', 'lat', 'long'], axes):
    kk.plot(kind='scatter', x=xcol, y='price', ax=ax, alpha=0.4, color='b')

In [ ]:
kk['price'].corr(kk['zipcode'])


In [ ]:
kk['zipcode'].value_counts()

In [ ]:
kk['price'].mean()

In [ ]:
s =kk.loc[kk['price'] > 540269, ['zipcode']]

In [ ]:
s.value_counts()

In [ ]:
a =kk.loc[kk['zipcode'] ==98006, ['price']]
a.mean()